# Element Vektörlerinin Oluşturulması

Bu bölümde element vektörlerini oluşturacağız. Bir önceki ders 7b'de bu vektörleri $\mathbf{f}$ denklem (7)'de ifade etmiştik. Bu vektörleri tekrar yazalım:

$$ \mathbf{f} = \int_{\partial \Omega}\big(\mathbf{N}k\nabla (\mathbf{N}\cdot\mathbf{T})\big) \cdot \mathbf{n} dS  + \int_{\Omega} \mathbf{N} \big(h_z T_{\infty})  d\Omega = \mathbf{f}_1 + \mathbf{f}_2   \tag{1}$$ 

Denklem (1)'de görüldüğü üzere 1. ifadenin integrali geometri sınırları boyunca, 2. ifadenin integrali ise geometrinin kendisi üzerinde uygulanıyor. 

Burada yine başlangıç olarak 4. elemandan başlayalım. Ders 7c'de tanımladığımız şekil fonksiyonlarını ve türevlerini tekrar tanımlayalım:

In [1]:
import sympy as sp
import numpy as np
from scipy import integrate
from sympy.utilities import lambdify

xi = sp.Symbol("xi")
eta = sp.Symbol("eta")
x_ = sp.symbols("x1,x2,x3,x4")
y_ = sp.symbols("y1,y2,y3,y4")
şekil = sp.Matrix([(1+xi)*(1+eta)/4,
                   (1-xi)*(1+eta)/4,
                   (1-xi)*(1-eta)/4,
                   (1+xi)*(1-eta)/4])

In [2]:
nokta_sayısı = 4

gradyen_xi = sp.diff(şekil, xi)
gradyen_eta = sp.diff(şekil, eta)

gradyen = sp.Matrix([gradyen_eta, gradyen_xi])

gradyen = gradyen.reshape(2,4)

c_şekil = sp.Matrix([0, 0])
for i in range(nokta_sayısı):
    c_şekil[0] += x_[i] * şekil[i]
    c_şekil[1] += y_[i] * şekil[i]
J = c_şekil.jacobian([xi, eta]).T

4 numaralı elemanı tekrar göz önüne getirelim:

<img src="Resimler/Ders7/7d_ElemanMesh.png" align="center" width= "600"/>

Ardından elemanın koordinatlarını tekrar tanımlayalım:

In [3]:
x_koord = [1,0,0,1] 
y_koord = [2,2,1,1]
    
for nokta in range(nokta_sayısı):
    J = J.subs(x_[nokta],x_koord[nokta]) 
    J = J.subs(y_[nokta],y_koord[nokta])

### 1. integralin hesaplanması

Bu integrali hesaplarken ders 4 ve ders 5'te kullanığımız penaltı metodunu kullanacağız. Bu durumda herhangi bir sınır üzerindeki herhangi bir nokta için aşağıdaki denklem, o noktadaki sınır koşulunu ifade edebilir:

$$
\begin{equation}
\alpha \frac{dT}{dx}(x,y) + \beta T(x,y) = \gamma.
\end{equation} \tag{2}
$$

In [11]:
k = 5 # W/m2.K
q_1 = 500 # W/m2
q_2 = 1000 # W/m2
alpha = [-k,-k,0,-k]
gamma = [(q_1+q_2),q_1,0,q_2]


t_z = 0.05 # m
l = 1 # m
A = t_z*l # m2

f_e_1 = np.zeros((nokta_sayısı,1))

for i in range(nokta_sayısı):
    
    if alpha[i] == 0:
        alpha[i] = 1e-12
    
    f_e_1[i] += gamma[i]*k*A/alpha[i]

f_e_1

array([[-75.],
       [-25.],
       [  0.],
       [-50.]])

### 2. integralin hesaplanması  

Denklem (1)'deki 2. terim olan

$$ \mathbf{f}_2 = \int_{\Omega} \mathbf{N} \big(h_z T_{\infty})  d\Omega $$

ifadesinin hesaplanması 1. terime kıyasla daha kolaydır. Bu ıfadeyi de yine izoparametrik eşleme ile yazarsak:

$$ \int_{\Omega} \mathbf{N} \big(h_z T_{\infty})  d\Omega = 
\int_{\Omega}
h_z T_{\infty}
\begin{bmatrix}
N_1 \\ N_2 \\ N_3 \\ N_4
\end{bmatrix}
d\Omega
=
\int_{-1}^1\int_{-1}^1
h_z T_{\infty}
\begin{bmatrix}
N_1 \\ N_2 \\ N_3 \\ N_4
\end{bmatrix}
det(\mathbf{J}) d\eta d\xi
$$

denklemini elde ederiz. Bu denklemi yine *sympy* ile yazalım:

In [5]:
h = 10 # W/m2K
t_z = 0.05 #m
h_z = 2*h/t_z
T_sonsuz = 30 # C

f_e_2_sym = h_z*T_sonsuz*şekil*J.det()
f_e_2_sym

Matrix([
[750.0*(eta + 1)*(xi + 1)],
[750.0*(1 - xi)*(eta + 1)],
[750.0*(1 - eta)*(1 - xi)],
[750.0*(1 - eta)*(xi + 1)]])

In [6]:
f_e_2 = np.zeros((nokta_sayısı,1))

for i in range(nokta_sayısı):
    geçici_fonksiyon = lambdify( (xi,eta), f_e_2_sym[i], 'math' )
    f_e_2[i] += integrate.nquad(geçici_fonksiyon, [[-1, 1],[-1, 1]])[0]

f_e_2

array([[3000.],
       [3000.],
       [3000.],
       [3000.]])